In [1]:
import geojson
import pandas as pd
import json
import pprint as pp

In [2]:
path = "Resources/VICTORIAN_ROAD_CRASH_DATA.geojson"

In [3]:
def geojson_to_dataframe(geojson_file):
    """
    Reads a GeoJSON file, extracts properties, and latitude and longitude from each feature,
    and converts them into a DataFrame.

    Args:
        geojson_file (str): Path to the GeoJSON file.

    Returns:
        pd.DataFrame: DataFrame containing properties, latitude, and longitude from the GeoJSON file.
    """
    # Read and parse the GeoJSON file
    with open(geojson_file, 'r') as file:
        data = json.load(file)
    
    # Ensure that the file has a 'features' key
    if 'features' not in data:
        raise ValueError("GeoJSON file does not contain 'features' key.")

    # Extract properties, latitude, and longitude from each feature
    features_list = []
    for feature in data['features']:
        properties = feature.get('properties', {})
        geometry = feature.get('geometry', {})
        
        # Default latitude and longitude values
        lat = lon = None
        
        # Handle Point geometries
        if geometry.get('type') == 'Point':
            coordinates = geometry.get('coordinates', [])
            if len(coordinates) == 2:
                lon, lat = coordinates  # GeoJSON uses (lon, lat) order
        
        # Combine properties, latitude, and longitude into a single dictionary
        feature_data = {**properties, 'LATITUDE': lat, 'LONGITUDE': lon}
        features_list.append(feature_data)
    
    # Convert list of features to DataFrame
    df = pd.DataFrame(features_list)

    return df


In [4]:
car_acc_data = geojson_to_dataframe(path)

In [5]:
car_acc_data.columns
print(len(car_acc_data))

170989


In [31]:
def clean_null_columns(df):
    """
    Function to remove/replace null values in a dataframe.
    
    Args:
        df (Dataframe): Specific dataframe.
        
    Returns:
        dataframe with the changes to null values.
    """
    #Identify columns with null values and append them into a list
    column_nulls = []
    column_nulls_values = []
    for item in df:
        if df[item].isna().sum() > 0:
            column_nulls.append(item)
            column_nulls_values.append(df[item].isna().sum())
            
    #Cobmine column nulls with column values into a dataframe
    null_columns_dict = {"column_names":column_nulls, "values":column_nulls_values}
    null_columns = pd.DataFrame(data=null_columns_dict).reset_index(drop=True)
    
    """
    Loop through each null column and replace/remove null values 
    """
    for column in null_columns["column_names"]:
        print(f"-------------------{column}--------------------------")
        print(f"{column} dtype is {df[column].dtype}")
        
        #Input for removing null values
        remove_input = input(f"{column}\nRemove nulls Y/N \n{df[column].unique()} ")
        
        #If statement for "Y" to removal
        if remove_input == "Y":
            #Remove null values
            print(f"Removing {null_columns.loc[null_columns['column_names'] == column, 'values'].values[0]}  rows")
            df = df.dropna(subset=[column])
        
        #Else if statement for "N"    
        elif remove_input == "N":
            #Input replace value
            replace_input = input("Replace input (replace value)/N ")
            
            # If replace input is not N, replace null with input
            if replace_input != "N":
                
                #If column dtype is numerical, then change input to an integer
                is_numeric =  (df[column].dtype == "float64") or (df[column].dtype == "int64")
                if is_numeric == True:
                    replace_input = int(replace_input)
            
                #Replace null value with input 
                df = df.fillna(value = {column:replace_input})
                
        #Print unique values after null value mangement         
        pp.pprint(df[column].unique())

    return df

In [32]:
def df_clean_check(df):
    """
    Function to go through the cleanning process of a dataframe
    
    Args:
        df (Dataframe): A specific dataframe
        
    Returns:
        Nothing.
    """
    print("-------------------DataFrame Info--------------------------")
    pp.pprint(df.info())
    
    #Identify missing values 
    print("-------------------Missing values--------------------------")
    print(df.isnull().sum())
    
    #Identify duplicates values 
    print("-------------------Duplicates--------------------------")
    print(df.duplicated().any().any())


In [33]:
df_clean_check(car_acc_data)

-------------------DataFrame Info--------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170989 entries, 0 to 170988
Data columns (total 52 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ACCIDENT_NO             170989 non-null  object 
 1   ACCIDENT_DATE           170989 non-null  object 
 2   ACCIDENT_TIME           170989 non-null  object 
 3   ACCIDENT_TYPE           170989 non-null  object 
 4   DAY_OF_WEEK             170989 non-null  object 
 5   DCA_CODE                170989 non-null  object 
 6   DCA_CODE_DESCRIPTION    170989 non-null  object 
 7   LIGHT_CONDITION         170989 non-null  object 
 8   POLICE_ATTEND           170989 non-null  object 
 9   ROAD_GEOMETRY           170989 non-null  object 
 10  SEVERITY                170989 non-null  object 
 11  SPEED_ZONE              170989 non-null  object 
 12  RUN_OFFROAD             170989 non-null  object 
 13  ROAD_NAME     

In [34]:
clean_data = clean_null_columns(car_acc_data)

-------------------NO_OF_VEHICLES--------------------------
NO_OF_VEHICLES dtype is float64
NO_OF_VEHICLES
Remove nulls Y/N 
[ 1.  2.  3. 13.  4.  5.  6.  8. 16.  7. 10.  9. 12. 11. 14. 21. nan 19.] N
Replace input (replace value)/N 0
array([ 1.,  2.,  3., 13.,  4.,  5.,  6.,  8., 16.,  7., 10.,  9., 12.,
       11., 14., 21.,  0., 19.])
-------------------HEAVYVEHICLE--------------------------
HEAVYVEHICLE dtype is float64
HEAVYVEHICLE
Remove nulls Y/N 
[ 0.  1.  2.  3.  4. nan] N
Replace input (replace value)/N 0
array([0., 1., 2., 3., 4.])
-------------------PASSENGERVEHICLE--------------------------
PASSENGERVEHICLE dtype is float64
PASSENGERVEHICLE
Remove nulls Y/N 
[ 1.  2.  0.  3. 13.  4.  5.  6.  8. 16.  7. 10.  9. 12. 19. nan 11. 17.] N
Replace input (replace value)/N 0
array([ 1.,  2.,  0.,  3., 13.,  4.,  5.,  6.,  8., 16.,  7., 10.,  9.,
       12., 19., 11., 17.])
-------------------MOTORCYCLE--------------------------
MOTORCYCLE dtype is float64
MOTORCYCLE
Remove nulls Y/

In [35]:
df_clean_check(clean_data)

-------------------DataFrame Info--------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170989 entries, 0 to 170988
Data columns (total 52 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ACCIDENT_NO             170989 non-null  object 
 1   ACCIDENT_DATE           170989 non-null  object 
 2   ACCIDENT_TIME           170989 non-null  object 
 3   ACCIDENT_TYPE           170989 non-null  object 
 4   DAY_OF_WEEK             170989 non-null  object 
 5   DCA_CODE                170989 non-null  object 
 6   DCA_CODE_DESCRIPTION    170989 non-null  object 
 7   LIGHT_CONDITION         170989 non-null  object 
 8   POLICE_ATTEND           170989 non-null  object 
 9   ROAD_GEOMETRY           170989 non-null  object 
 10  SEVERITY                170989 non-null  object 
 11  SPEED_ZONE              170989 non-null  object 
 12  RUN_OFFROAD             170989 non-null  object 
 13  ROAD_NAME     

In [44]:
def convert_column_to_datetime(df, column_name, output_format='%Y-%m-%d', date_format='%Y%m%d'):
    """
     Args:
        df (Dataframe): A specific dataframe
        
        column_name (string): The name of the column which is to be converted to datetime format.
        
        output_format (string): The desired output format of the datetime column.
        
        date_format (string): The format of the input dates in the column.
        
    Returns:
        dataframe with the changed date format.
    """
    # Converts specific column to datetime
    df[column_name] = pd.to_datetime(df[column_name], format=date_format, errors='coerce')
    
    # Formart datetime column into desired format
    df[column_name] = df[column_name].dt.strftime(output_format)

    return df

In [48]:
converted_df = convert_column_to_datetime(clean_data, 'ACCIDENT_DATE', output_format='%d/%m/%Y', date_format='%Y%m%d')

In [49]:
df_clean_check(converted_df)

-------------------DataFrame Info--------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170989 entries, 0 to 170988
Data columns (total 52 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ACCIDENT_NO             170989 non-null  object 
 1   ACCIDENT_DATE           170989 non-null  object 
 2   ACCIDENT_TIME           170989 non-null  object 
 3   ACCIDENT_TYPE           170989 non-null  object 
 4   DAY_OF_WEEK             170989 non-null  object 
 5   DCA_CODE                170989 non-null  object 
 6   DCA_CODE_DESCRIPTION    170989 non-null  object 
 7   LIGHT_CONDITION         170989 non-null  object 
 8   POLICE_ATTEND           170989 non-null  object 
 9   ROAD_GEOMETRY           170989 non-null  object 
 10  SEVERITY                170989 non-null  object 
 11  SPEED_ZONE              170989 non-null  object 
 12  RUN_OFFROAD             170989 non-null  object 
 13  ROAD_NAME     

In [51]:
converted_df["ACCIDENT_DATE"]

0         01/01/2012
1         01/01/2012
2         01/01/2012
3         01/01/2012
4         01/01/2012
             ...    
170984    31/12/2023
170985    31/12/2023
170986    31/12/2023
170987    31/12/2023
170988    31/12/2023
Name: ACCIDENT_DATE, Length: 170989, dtype: object

In [52]:
converted_df.head()

,ACCIDENT_NO,ACCIDENT_DATE,ACCIDENT_TIME,ACCIDENT_TYPE,DAY_OF_WEEK,DCA_CODE,DCA_CODE_DESCRIPTION,LIGHT_CONDITION,POLICE_ATTEND,ROAD_GEOMETRY,...,NO_OF_VEHICLES,HEAVYVEHICLE,PASSENGERVEHICLE,MOTORCYCLE,PT_VEHICLE,DEG_URBAN_NAME,SRNS,RMA,DIVIDED,STAT_DIV_NAME
0,T20120000009,01/01/2012,022500,Collision with a fixed object,Sunday,171,LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE,Dark No street lights,Yes,Not at intersection,...,1.0,0.0,1.0,0.0,0.0,RURAL_VICTORIA,C,Arterial Other,Undivided,Country
1,T20120000012,01/01/2012,020000,Collision with vehicle,Sunday,110,CROSS TRAFFIC(INTERSECTIONS ONLY),Dark Street lights on,Yes,Cross intersection,...,2.0,0.0,2.0,0.0,0.0,MELB_URBAN,UNKNOWN,UNKNOWN,UNKNOWN,Metro
2,T20120000013,01/01/2012,033500,Collision with vehicle,Sunday,160,VEHICLE COLLIDES WITH VEHICLE PARKED ON LEFT O...,Dark Street lights on,Yes,T intersection,...,2.0,0.0,2.0,0.0,0.0,MELB_URBAN,UNKNOWN,Arterial Other,Undivided,Metro
3,T20120000018,01/01/2012,051500,Collision with a fixed object,Sunday,173,RIGHT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE,Dark No street lights,Yes,Cross intersection,...,1.0,0.0,1.0,0.0,0.0,RURAL_VICTORIA,A,Arterial Highway,Undivided,Country
4,T20120000021,01/01/2012,073000,Collision with a fixed object,Sunday,171,LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE,Day,Yes,Not at intersection,...,1.0,0.0,1.0,0.0,0.0,MELB_URBAN,UNKNOWN,Local Road,Undivided,Metro
